In [22]:
import_word = "C:/Users/richi/Downloads/IAASB-International-Standard-Sustainability-5000-Exposure-Draft_0.docx"
import_pdf = "C:/Users/richi/Downloads/Deep_Learning___Assignment_5c.pdf"

In [7]:
# ! pip install openai
# ! pip install os
# ! pip install pypdf 
# ! pip install docx2txt
# ! pip install unstructured
# ! pip install python-docx


In [8]:
import openai
import os
from dotenv import load_dotenv, find_dotenv

# Get openai_api_key using .env file
_ = load_dotenv(find_dotenv("C:/Users/richi/OneDrive/Documents/OpenAI API practice/openai_api_key.env")) # .env filepath
openai.api_key = os.environ["OPENAI_API_KEY"]

In [9]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA,  ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader

# necessary for GUI / dashboard
import panel as pn  
pn.extension()

# Get recent LLM version
import datetime
current_date = datetime.datetime.now().date()
llm_name = "gpt-3.5-turbo-0301" if current_date < datetime.date(2023, 9, 2) else "gpt-3.5-turbo"

**Check document formatting alternatives**

- PDF: more informative chunks
- Word: preserves structure of data better

In [23]:
# # Import as pdf (with meta data elaboration)
# loader = PyPDFLoader(import_pdf)
# doc = loader.load()

# # for i in range(len(doc)):
# #     print(doc[i])
# #     print("\n\n")

# # split documents
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
# doc = text_splitter.split_documents(doc)

# Import as word docx (with meta data elaboration)
from langchain.document_loaders import UnstructuredWordDocumentLoader
loader = UnstructuredWordDocumentLoader(import_word, mode = "elements")
doc = loader.load()

# Only keep chunk if the content length is large (short sentences like titles usually have little to no semantic value)
doc = [d for d in doc if len(d.page_content) > 100]

print("---------- len doc: ----", len(doc))

for i in range(len(doc)):
    print(doc[i])
    print("\n\n")

---------- len doc: ---- 1690
page_content='This document has been prepared and approved by the International Auditing and Assurance Standards Board. It does not constitute an authoritative pronouncement of the IAASB, nor does it amend, extend or override the International Standards on Auditing or other of the IAASB’s International Standards.' metadata={'source': 'C:/Users/richi/Downloads/IAASB-International-Standard-Sustainability-5000-Exposure-Draft_0.docx', 'filename': 'IAASB-International-Standard-Sustainability-5000-Exposure-Draft_0.docx', 'file_directory': 'C:/Users/richi/Downloads', 'last_modified': '2023-08-15T15:29:23', 'filetype': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document', 'category': 'NarrativeText'}



page_content='The objective of the IAASB is to serve the public interest by setting high-quality auditing, assurance, and other related standards and by facilitating the convergence of international and national auditing and assurance standards

## LLM Pipeline

In [24]:
# Function to load database
def load_db(file, chain_type, k):
    # load documents
#     loader = PyPDFLoader(file)
#     documents = loader.load()
    
    loader = UnstructuredWordDocumentLoader(import_word, mode = "elements")
    documents = loader.load()

    # Only keep chunk if the content length is large (short sentences like titles usually have little to no semantic value)
    documents = [d for d in doc if len(d.page_content) > 100]
    
    # split documents
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_documents(documents)
    
    # define embedding
    embeddings = OpenAIEmbeddings()
    
    # create vector database from data
    db = DocArrayInMemorySearch.from_documents(docs, embeddings)
    
    # define retriever
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})
    
    # create a chatbot chain. Memory is managed externally.
    qa = ConversationalRetrievalChain.from_llm(
        llm=ChatOpenAI(model_name=llm_name, temperature=0), # define LLM (name ie e.g. "gpt-3.5-turbo")
        chain_type=chain_type, 
        retriever=retriever, 
        return_source_documents=True,
        return_generated_question=True,
    )
    return qa 

## Build GUI:

In [25]:
import panel as pn
import param

# Class to construct Q&A
class cbfs(param.Parameterized):
    chat_history = param.List([])
    answer = param.String("")
    db_query  = param.String("")
    db_response = param.List([])
    
    def __init__(self,  **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.loaded_file = import_pdf # input document is passed in here
        self.qa = load_db(self.loaded_file,"stuff", 4)
    
    def call_load_db(self, count):
        if count == 0 or file_input.value is None:  # init or no file specified :
            return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")
        else:
            file_input.save("temp.pdf")  # local copy
            self.loaded_file = file_input.filename
            button_load.button_style="outline"
            self.qa = load_db("temp.pdf", "stuff", 4)
            button_load.button_style="solid"
        self.clr_history()
        return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")

    def convchain(self, query):
        if not query:
            return pn.WidgetBox(pn.Row('User:', pn.pane.Markdown("", width=600)), scroll=True)
        result = self.qa({"question": query, "chat_history": self.chat_history}) # Passing in chat history
        self.chat_history.extend([(query, result["answer"])]) # Extending query with result of context
        self.db_query = result["generated_question"]
        self.db_response = result["source_documents"]
        self.answer = result['answer'] 
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=600)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=600, style={'background-color': '#F6F6F6'}))
        ])
        inp.value = ''  #clears loading indicator when cleared
        return pn.WidgetBox(*self.panels,scroll=True)

    @param.depends('db_query ', )
    def get_lquest(self):
        if not self.db_query :
            return pn.Column(
                pn.Row(pn.pane.Markdown(f"Last question to DB:", styles={'background-color': '#F6F6F6'})),
                pn.Row(pn.pane.Str("no DB accesses so far"))
            )
        return pn.Column(
            pn.Row(pn.pane.Markdown(f"DB query:", styles={'background-color': '#F6F6F6'})),
            pn.pane.Str(self.db_query )
        )

    @param.depends('db_response', )
    def get_sources(self):
        if not self.db_response:
            return 
        rlist=[pn.Row(pn.pane.Markdown(f"Result of DB lookup:", styles={'background-color': '#F6F6F6'}))]
        for doc in self.db_response:
            rlist.append(pn.Row(pn.pane.Str(doc)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    @param.depends('convchain', 'clr_history') 
    def get_chats(self):
        if not self.chat_history:
            return pn.WidgetBox(pn.Row(pn.pane.Str("No History Yet")), width=600, scroll=True)
        rlist=[pn.Row(pn.pane.Markdown(f"Current Chat History variable", styles={'background-color': '#F6F6F6'}))]
        for exchange in self.chat_history:
            rlist.append(pn.Row(pn.pane.Str(exchange)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    def clr_history(self,count=0):
        self.chat_history = []
        return 

In [26]:
cb = cbfs()

file_input = pn.widgets.FileInput(accept= '.docx') #chanege which type is accepts
button_load = pn.widgets.Button(name="Load DB", button_type='primary')
button_clearhistory = pn.widgets.Button(name="Clear History", button_type='warning')
button_clearhistory.on_click(cb.clr_history)
inp = pn.widgets.TextInput( placeholder='Enter text here…')

bound_button_load = pn.bind(cb.call_load_db, button_load.param.clicks)
conversation = pn.bind(cb.convchain, inp) 

jpg_pane = pn.pane.Image( './img/convchain.jpg')

tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=300),
    pn.layout.Divider(),
)
tab2= pn.Column(
    pn.panel(cb.get_lquest),
    pn.layout.Divider(),
    pn.panel(cb.get_sources ),
)
tab3= pn.Column(
    pn.panel(cb.get_chats),
    pn.layout.Divider(),
)
tab4=pn.Column(
    pn.Row( file_input, button_load, bound_button_load),
    pn.Row( button_clearhistory, pn.pane.Markdown("Clears chat history. Can use to start a new topic" )),
    pn.layout.Divider(),
    pn.Row(jpg_pane.clone(width=400))
)
dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('#IAASB LLM Interface')),
    pn.Tabs(('Conversation', tab1), ('Database', tab2), ('Chat History', tab3),('Configure', tab4))
)

# Launch GUI
dashboard

Column
    [0] Row
        [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Row
                [0] TextInput(placeholder='Enter text here…')
            [1] Divider()
            [2] ParamFunction(function, _pane=WidgetBox, height=300, loading_indicator=True)
            [3] Divider()
        [1] Column
            [0] ParamMethod(method, _pane=Column)
            [1] Divider()
            [2] ParamMethod(method, _pane=Str)
        [2] Column
            [0] ParamMethod(method, _pane=WidgetBox)
            [1] Divider()
        [3] Column
            [0] Row
                [0] FileInput(accept='.docx')
                [1] Button(button_type='primary', name='Load DB')
                [2] ParamFunction(function, _pane=Markdown)
            [1] Row
                [0] Button(button_type='warning', name='Clear History')
                [1] Markdown(str)
            [2] Divider()
            [3] Row
                [0] Image(str, width=400)